In [16]:
import distutils.util
import subprocess


# Configure MuJoCo to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

import mujoco

#@title Other imports and helper functions
import numpy as np
from typing import Callable, Optional, Union, List
import scipy.linalg

# Graphics and plotting.
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# Import Renderer Class
import os
import sys

from controller import Controller

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from renderer import Renderer
model = mujoco.MjModel.from_xml_path('model/world.xml') #cartpole.xml') #humanoid.xml
data = mujoco.MjData(model)
renderer = Renderer(model, width=640, height=480)

# Make a new camera, move it to a closer distance.
camera = mujoco.MjvCamera()
camera_id = camera
mujoco.mjv_defaultFreeCamera(model, camera)
#camera.type = mujoco.mjtCamera.mjCAMERA_FIXED
#camera.fixedcamid = 0
camera.lookat = (0, 0, 0)
camera.distance = 3
camera.elevation = -90 
#camera.trackbodyid = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY , "Kugel")

DURATION  = 10   # seconds
FRAMERATE = 60  # Hz

# Controller übernimmt resest Optionen und startet später Optimierung
controller = Controller(model, data)

data.ctrl[0] = 5
model.body("arm").pos = controller.setstartingposarm()
data.joint("rotforce").qpos = controller.setstartingposball()

frames = []

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
#scene_option


env: MUJOCO_GL=egl


Von wo soll der Arm gestartet werden. Geben Sie die y Koordinante ein!
 Sie können dabeiWerte zwischen -0.53 und 0.53! 0.35


In [17]:
i=-1
while data.time < DURATION:
  # Step the simulation.
    mujoco.mj_step(model, data)
    if data.time > 1 and i == -1:
        data.ctrl[model.actuator("schwung").id] = 3 
        i = i + 1
    if data.time > 3 and i == 0 :
        data.ctrl[model.actuator("schwung").id] = controller.setstartingctrl()
        i = i + 1
    if data.qpos[model.joint("rotforce").qposadr] < 19.2 and i == 1:
        data.ctrl[0] = 0
        data.ctrl[1] = 0
        i = i + 1 
    
        
# Render and save frames.    
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data, camera)#,scene_option for debugging
        pixels = renderer.render()
        frames.append(pixels.copy())

media.show_video(frames, fps=FRAMERATE)


Geben Sie ein wie viel Kraft sie auf die Kugel bringen möchten!
 Sie können dabeiWerte zwischen -100 und -1 eingeben! -10


This browser does not support the video tag.


In [2]:
data.joint("rotforce").qpos

array([19.2  ,  0.53 ,  0.109,  1.   ,  0.   ,  0.   ,  0.   ])